# Import Library Package

In [1]:
import requests # 인터넷에서 Data를 가져오기 위한 Library
from bs4 import BeautifulSoup # 웹 페이지 내용을 분석하기 위한 Library
import time # 대기 시간을 추가하기 위한 Library
import random # Random한 대기 시간을 만들기 위한 Library
from tqdm import tqdm # Crawling 진행 상황을 체크하기 위한 Module

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import pandas as pd

from konlpy.tag import Okt
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from transformers import pipeline, AutoTokenizer, TFAutoModelForSequenceClassification

from pathlib import Path

In [7]:
# # Chrome Browser와 Chrome Driver Version 확인하기
# chrome_options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = chrome_options)
# driver.get("https://news.naver.com/main/ranking/popularDay.naver")

# # 기본 File Name 설정
# base_filename = "NAVER News - 언론사별 랭킹뉴스"
# extension = ".png"
# index = 1  # File Name에 추가될 숫자
# new_filename = base_filename + extension

# # 동일한 File Name이 존재하는지 확인하고, 존재한다면 새로운 File Name 생성
# while Path(f"../Data/ScreenShot/{new_filename}").exists() :
#   new_filename = f"{base_filename}_{index}{extension}"
#   index += 1

# # 현재 화면 Capture하기
# driver.save_screenshot(f"../Data/ScreenShot/{new_filename}")

True

# NAVER News - 언론사별 랭킹뉴스 Crawling

## 언론사, Title, Link 추출하기

In [8]:
# 뉴스 Crawling
def get_news_links_by_press (url) :
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
  response = requests.get(url, headers = headers)
  soup = BeautifulSoup(response.content, 'html.parser')
  
  press_data = {}
  press_sections = soup.select('.rankingnews_box')
  
  for press_section in tqdm(press_sections, desc = "언론사별 뉴스 크롤링") :
    press_name = press_section.select_one('.rankingnews_name').get_text(strip = True)
    news_links = set()  # 중복 제거를 위한 set 사용
    for item in press_section.select('li a') :
      title = item.get_text(strip = True)
      link = item['href']
      if title and link and "동영상" not in title :  # Title이 존재하고 "동영상"이 포함되지 않은 경우에만 추가
        news_links.add((title, link))
    press_data[press_name] = list(news_links)[:5]  # 다시 list로 변환 후 상위 5개만 저장
    
    # 각 언론사별 뉴스 Crawling 후 대기 시간 추가
    time.sleep(random.uniform(0.5, 2.0))
  
  return press_data

In [9]:
base_url = 'https://news.naver.com/main/ranking/popularDay.naver'
press_news_data = get_news_links_by_press(base_url)

# 뉴스 DataFrame 생성
news_list = []
for press_name, news_data in press_news_data.items() :
  for title, link in news_data :
    news_list.append([press_name, title, link])
df = pd.DataFrame(news_list, columns = ['Press', 'Title', 'Link'])

df

언론사별 뉴스 크롤링: 100%|██████████| 82/82 [01:46<00:00,  1.30s/it]


,Press,Title,Link
0,매일경제,“美명문 로스쿨 합격했다”…인기 많던 ‘재벌 4세’ 패션女의 정체,https://n.news.naver.com/article/009/000531693...
1,매일경제,"SBS 개그맨, 돌연 김호중 소속사 대표와 정찬우 고소…알고 보니",https://n.news.naver.com/article/009/000531710...
2,매일경제,"“물린 내 주식 대체 어떡하라고”…뿔난 개미 5만명, 금투세 폐지 청원",https://n.news.naver.com/article/009/000531695...
3,매일경제,백종원이 한그릇 후루룩 해치웠더니…너도 나도 “뜨거운 비빔면”,https://n.news.naver.com/article/009/000531700...
4,매일경제,40대 교사·여고생 제자 관계에…대전 학부모·학교 발칵,https://n.news.naver.com/article/009/000531704...
...,...,...,...
399,더스쿠프,"""쿠팡서 4배, 야놀자서 1.5배"" 소뱅, 라인으론 얼마 챙길까",https://n.news.naver.com/article/665/000000313...
400,더스쿠프,소문난 '밸류업'에 먹을 건 없었다 [視리즈],https://n.news.naver.com/article/665/000000314...
401,더스쿠프,집중호우 2년 後 … 반지하에선 '위험'이 빠지지 않았다 [추적+],https://n.news.naver.com/article/665/000000313...
402,레이디경향,"청순함에 세련미까지 더한, 송혜교의 공항 패션",https://n.news.naver.com/article/145/000002084...


## 본문 수집하기

In [23]:
# Chrome Browser와 Chrome Driver Version 확인 및 WebDriver 객체 생성
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = chrome_options)


# tqdm Module을 이용하여 진행 상황 표시
pbar = tqdm(df['Link'][:10], desc = "크롤링 진행 중", unit = "링크")

# 뉴스 기사 Link를 순회하며 기사 내용 추출
articles = []
for link in pbar :
    driver.get(link)

    # 기본 File Name 설정
    base_filename = df[df['Link'] == link]['Title'].values[0]
    extension = ".png"
    index = 1  # File Name에 추가될 숫자
    new_filename = base_filename + extension

    # 동일한 File Name이 존재하는지 확인하고, 존재한다면 새로운 File Name 생성
    while Path(f"../Data/ScreenShot/{new_filename}").exists() :
        new_filename = f"{base_filename}_{index}{extension}"
        index += 1

    # 현재 화면 Capture하기
    driver.save_screenshot(f"../Data/ScreenShot/{new_filename}")

    html = driver.page_source
    article_soup = BeautifulSoup(html, "html.parser")
    content = article_soup.select_one("#contents")
    if content :
        # 공백과 HTML Tag 제거
        text = ' '.join(content.text.split())
        articles.append(text)
    time.sleep(2)  # 페이지 로드를 기다립니다.
    # # 요청 후 임의의 시간만큼 대기
    # time.sleep(random.uniform(0.5, 2.0))

# Browser 종료
driver.quit()

# 빈 문서가 있는지 확인
articles = [article for article in articles if article.strip()]

# 기사 내용이 제대로 수집되었는지 확인
for i, article in enumerate(articles) :
    print(f"기사 {i + 1}: {article[:100]}...")  # 기사 내용 앞부분만 출력

# NLTK를 이용하여 불용어 제거, 단어 토큰화, 표제어 추출
## 한국어 불용어 모음집 불러오기
stopword_list = pd.read_csv("../Data/stopword.txt", header = None)
stopword_list[0] = stopword_list[0].apply(lambda x: x.strip())
korean_stopwords = stopword_list[0].to_numpy()

nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
tokenized_articles = []
for article in tqdm(articles, desc = "텍스트 처리 중", unit = "기사") :
    tokens = word_tokenize(article)
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.isalnum() and word.lower() not in korean_stopwords]
    tokenized_articles.append(' '.join(tokens))

# 빈 문서가 있는지 다시 확인
tokenized_articles = [article for article in tokenized_articles if article.strip()]

# 각 기사별 Keyword 추출 (빈도 높은 단어)
keywords = []
for article in tokenized_articles :
    word_counts = Counter(article.split())
    common_words = word_counts.most_common(10)  # 상위 10개 단어 추출
    keywords.append([word for word, freq in common_words])

# Keyword 확인
for i, kw in enumerate(keywords) :
    print(f"기사 {i + 1} 키워드: {kw}")

크롤링 진행 중: 100%|██████████| 10/10 [00:36<00:00,  3.60s/링크]
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gwangyeong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gwangyeong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


기사 1: 패션 인플루언서로 활동하던 DL그룹(옛 대림그룹) 오너가 4세 이주영(24)씨가 최근 미국 법학대학원(로스쿨)에 합격한 것으로 알려졌다. [사진출처 = 인스타그램]패션 인플루언서로...
기사 2: 영장실질심사를 마치고 유치장으로 이동하는 이광득 대표. 사진ㅣ유용석 기자가수 김호중 소속사가 ‘음주 뺑소니’ 사건에 이어 주식 취득 관련 소송전까지 휘말리며 연이은 악재에 휩싸였다...
기사 3: 22대 국회서 ‘금투세 폐지’ 심사요건 충족청원인 “기계적 부자 감세 황당, 역차별” [사진 이미지 = 챗 GPT]내년 시행을 앞둔 금융투자소득세(금투세) 폐지와 관련한 국회 국민...
기사 4: 라면 제조사 ‘컵 비빔면’ 잇달아 출시“편의점 판매 경쟁력 확보 위한 전략”컵 비빔면 활용한 ‘뜨빔면’ 유행 (왼쪽부터) 팔도비빔면 용기면, 더미식비빔면 용기면, 배홍동 큰사발면,...
기사 5: [사진출처 = 연합뉴스]대전에 위치한 한 고등학교에서 40대 체육 교사가 여제자와 부적절한 교제를 한 것으로 알려져 시교육청이 조사에 나섰다.11일 대전시교육청에 따르면 학교에서 ...
기사 6: 변호인 “박세리 父, 재단 법인 도장 무단 제작‧사용” 골프선수 출신 감독 겸 방송인 박세리씨가 이끄는 박세리희망재단이 박씨의 부친을 사문서위조 혐의로 경찰에 고소했다. 박세리 전...
기사 7: 이정재 뛰어들었으나 큐캐피탈이 인수 유력초록뱀 인수 두고 이정재-래몽래인 경영권 분쟁 벌여 초록뱀미디어 CI. /초록뱀미디어 제공 드라마 ‘나의 아저씨’와 ‘나의 해방일지’, ‘펜...
기사 8: 정보통신망법 위반으로 고소 당해 시민 331명, 강씨 부부 고발장 우편 발송 반려견 훈련사 강형욱 보듬컴퍼니 대표와 아내 수잔 엘더 이사가 전 직원들로부터 고소를 당했다. 강형욱 ...
기사 9: 중국 지린성(省)에서 미국인 대학 강사 4명이 괴한이 휘두른 흉기에 찔리는 사건이 발생했다. 피해자들은 모두 지린성 베이화대학과의 학술 교류를 위해 중국에 머물렀던 것으로 알려졌다...
기사 10: 한국시

텍스트 처리 중: 100%|██████████| 10/10 [00:00<00:00, 95.68기사/s]

기사 1 키워드: ['구독', '0', '닫기', '미국', '수', '기자', '매일경제', '있습니다', '언론사', '패션']
기사 2 키워드: ['구독', '닫기', '이광득', '수', '대표와', '기자', '매일경제', '있습니다', '언론사', '관련']
기사 3 키워드: ['금투세', '수', '구독', '닫기', '있습니다', '폐지', '기자', '매일경제', '언론사', '22대']
기사 4 키워드: ['비빔면', '컵라면을', '뜨빔면', '수', '구독', '닫기', '용기면', '차갑게', '젊은', '뜨거운']
기사 5 키워드: ['구독', '0', '닫기', '수', '해당', '기자', '매일경제', '있습니다', '언론사', '기사를']
기사 6 키워드: ['구독', '0', '닫기', '박세리', '수', '기자', '있습니다', '언론사', '고소했다', '혐의']
기사 7 키워드: ['구독', '초록뱀미디어', '수', '닫기', '큐캐피탈이', '경영권', '것으로', '기자', '있습니다', '언론사']
기사 8 키워드: ['구독', '닫기', '강형욱', '대한', '수', '유튜브', '메신저', '기자', '있습니다', '언론사']
기사 9 키워드: ['구독', '수', '닫기', '중국', '지린성', '강사', '발생했다', '위해', '것으로', '기자']
기사 10 키워드: ['수', '구독', '푸바오', '닫기', '중국', '볼', '기자', '있습니다', '언론사', '0']


In [ ]:
# # tqdm을 이용하여 진행 상황 표시
# pbar = tqdm(df['Link'][:10], desc = "크롤링 진행 중", unit = "링크")

# # 뉴스 기사 Link를 순회하며 기사 내용 추출
# articles = []
# for link in pbar :
#     article_response = requests.get(link)
#     article_html = article_response.text
#     article_soup = BeautifulSoup(article_html, "html.parser")
#     content = article_soup.select_one("#articleBodyContents")
#     if content :
#         # 공백과 HTML Tag 제거
#         text = ' '.join(content.text.split())
#         articles.append(text)
#     # 요청 후 임의의 시간만큼 대기
#     time.sleep(random.uniform(0.5, 2.0))

# # 빈 문서가 있는지 확인
# articles = [article for article in articles if article.strip()]

# # 기사 내용이 제대로 수집되었는지 확인
# for i, article in enumerate(articles) :
#     print(f"기사 {i+1}: {article[:100]}...")  # 기사 내용 앞부분만 출력

# # NLTK를 이용하여 불용어 제거, 단어 토큰화, 표제어 추출
# ## 한국어 불용어 모음집 불러오기
# stopword_list = pd.read_csv("../Data/stopword.txt", header = None)
# stopword_list[0] = stopword_list[0].apply(lambda x: x.strip())
# korean_stopwords = stopword_list[0].to_numpy()

# nltk.download('punkt')
# nltk.download('wordnet')
# lemmatizer = WordNetLemmatizer()
# tokenized_articles = []
# for article in tqdm(articles, desc = "텍스트 처리 중", unit = "기사") :
#     tokens = word_tokenize(article)
#     tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.isalnum() and word.lower() not in korean_stopwords]
#     tokenized_articles.append(' '.join(tokens))

# # 빈 문서가 있는지 다시 확인
# tokenized_articles = [article for article in tokenized_articles if article.strip()]

# # 각 기사별 Keyword 추출 (빈도 높은 단어)
# keywords = []
# for article in tokenized_articles :
#     word_counts = Counter(article.split())
#     common_words = word_counts.most_common(10)  # 상위 10개 단어 추출
#     keywords.append([word for word, freq in common_words])

# # Keyword 확인
# for i, kw in enumerate(keywords) :
#     print(f"기사 {i + 1} 키워드: {kw}")